In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

Downloading:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/953 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/851k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/638M [00:00<?, ?B/s]

In [ ]:
tokens = tokenizer.encode('It was good but couldve been better. Great', return_tensors='pt')

In [ ]:
result = model(tokens)

In [ ]:
result.logits

tensor([[-2.7768, -1.2353,  1.4419,  1.9804,  0.4584]],
       grad_fn=<AddmmBackward>)

In [ ]:
int(torch.argmax(result.logits))+1

4

In [ ]:
r = requests.get('https://www.yelp.com/biz/cafe-la-trova-miami')
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*comment.*')
results = soup.find_all('p', {'class':regex})
reviews = [result.text for result in results]

In [ ]:
reviews

['The first time in this place.was a wonderful experience. Drinks and food excellent. our waitress was very nice and help us with the tapas and drinks. Highly recommend this place',
 'Thank you for the glowing review, Clara! You made our day :).',
 "If you're short on time...this was a great place in every way, food, drinks, decor, service and entertainment.Band was amazing and they placed us close..Started with Papa Doble and wife had mojito. Recommend both, but mojito may need more sugar for some. \xa0I personally only order a mojito, when they have sugar cane stick in the mix...Started with Beet Salad (i hate beets and i loved this..with the nuts and the arugula(bitter)/cheese(salt) balancing the sweetness), Caesar salad (beautiful and heavy on the anchovy) and gambas (recommended, but different than spanish al ajillo, many beans, thicker sauce, great taste).For mains we had the snapper on the menu, arroz con pollo, and ropa vieja...all amazing, different and go together...2nd round

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df = pd.DataFrame(np.array(reviews), columns=['review'])

In [ ]:
df['review'].iloc[0]

'The first time in this place.was a wonderful experience. Drinks and food excellent. our waitress was very nice and help us with the tapas and drinks. Highly recommend this place'

In [ ]:
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [ ]:
sentiment_score(df['review'].iloc[1])

5

In [ ]:
df['sentiment'] = df['review'].apply(lambda x: sentiment_score(x[:512]))

In [ ]:
df

,review,sentiment
0,The first time in this place.was a wonderful e...,5
1,"Thank you for the glowing review, Clara! You m...",5
2,If you're short on time...this was a great pla...,4
3,Hi Harris! Thank you so much for taking the ti...,5
4,I came to cafe La trova for dinner with my wif...,3
5,"Hi Anant, Thank you so much for taking the tim...",5
6,A dear friend of mine was visiting from Cali a...,5
7,Hello Julio! Thank you so much for sharing yo...,5
8,Great Cuban food. This place was recommended t...,5
9,Thank you for taking the time to share this re...,5


In [ ]:
df['review'].iloc[4]

"I came to cafe La trova for dinner with my wife. Overall it was a fun experience, but was left a bit disappointed with the food / service. Will caveat this with I'm vegetarian (wife is not) and cafe La trova is definitely not vegetarian friendly.Food 2.5/5: I ordered a bunch of vegetarian dishes (pumpkin empanadas, mushroom croquettes, spinach croquettes, fried cheese, and beet salad) and my wife got the gambas Al ajillo. The empanadas were great but the rest of the vegetarian dishes were somewhere between mediocre and decent. My wife said her dish was good though so possible they just don't do the vegetarian dishes well.Service / ambience 4/5: The service wasn't great - our server rushed us on the order and wasn't very helpful when we asked questions about the menu. The ambience was great though - we came Monday night for dinner and they had live music playing. The restaurant had a great crowd and some people got up and started dancing to the music!"